In [1]:
include("../src/Julia.jl")

objc[10059]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[10059]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[10059]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[10059]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/se_iu_2/"

"../../graphs/se_iu_2/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [5]:
tree = []
for i in 1:3
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [6]:
x = readFromFile(graphp * "x.vec");

In [7]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [8]:
numIts = 200;

for i in 1:3
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  2.428402 seconds (275.03 k allocations: 11.676 MB, 0.56% gc time)
  1.270507 seconds (123.82 k allocations: 4.983 MB)
  5.298389 seconds (3.17 M allocations: 95.355 MB, 9.93% gc time)
Working on iteration 10 with a-norm error 35.25613677400866
Working on iteration 20 with a-norm error 14.073459347127468
Working on iteration 30 with a-norm error 5.335833560598752
Working on iteration 40 with a-norm error 2.3318233561812094
Working on iteration 50 with a-norm error 0.7149790191225619
Working on iteration 60 with a-norm error 0.2190165137717518
Working on iteration 70 with a-norm error 0.06982067742058587
Working on iteration 80 with a-norm error 0.02503757207263197
Working on iteration 90 with a-norm error 0.008694904988290935
Working on iteration 100 with a-norm error 0.003070653158206653
Working on iteration 110 with a-norm error 0.0009860804860165574
Working on iteration 120 with a-norm error 0.00028817131121742736
Working on iteration 130 with a-norm error 9.851440430730404e-5
Work